In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import csv
from scipy import stats
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

In [ ]:
TRAIN_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv"
TEST_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_single_test.tsv"
train_filepath = "/content/sample_data/train.csv"
test_filepath = "/content/sample_data/test.csv"

In [ ]:
k = 1            # token append number

In [ ]:
df_train = pd.read_csv(TRAIN_DATAPATH, sep = '\t', quotechar="'", quoting = csv.QUOTE_NONE)
df_test = pd.read_csv(TEST_DATAPATH, sep = '\t', quotechar="'", quoting = csv.QUOTE_NONE)
df_train['complexity'] = df_train['complexity'].astype(float)
df_test['complexity'] = df_test['complexity'].astype(float)
for i in range(len(df_train)):
    first = str(df_train['token'][i]) + " [SEP] "
    last = " [SEP] " + str(df_train['token'][i])
    for _ in range(k):
        df_train['sentence'][i] = first + df_train['sentence'][i] + last
for i in range(len(df_test)):
    first = str(df_test['token'][i]) + " [SEP] "
    last = " [SEP] " + str(df_test['token'][i])
    for _ in range(k):
        df_test['sentence'][i] = first + df_test['sentence'][i] + last
df_train = df_train.drop(['id', 'corpus', 'token'], axis = 1)
df_test = df_test.drop(['id', 'corpus', 'token'], axis = 1)
df_train = df_train[['complexity', 'sentence']]
df_test = df_test[['complexity', 'sentence']]
df_train.to_csv(train_filepath, index = False)
df_test.to_csv(test_filepath, index = False)

In [ ]:
device = 'cuda'
batch_size = 8
num_epochs = 4

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

In [ ]:
label = Field(sequential = False, use_vocab = False, batch_first = True, dtype = torch.float32)
text = Field(use_vocab = False, tokenize = tokenizer.encode, lower = False, batch_first = True, pad_token = PAD_INDEX, unk_token = UNK_INDEX)
fields = [('complexity', label), ('sentence', text)]
train = TabularDataset(path = train_filepath, format = 'csv', skip_header = True, fields = fields)
train_iter = BucketIterator(train, batch_size = batch_size, device = device, sort_key = lambda x: len(x.sentence), train = True)

In [ ]:
test_label = Field(sequential = False, use_vocab = False, batch_first = True, dtype = torch.float32)
test_text = Field(use_vocab = False, tokenize = tokenizer.encode, lower = False, batch_first = True, pad_token = PAD_INDEX, unk_token = UNK_INDEX)
test_fields = [('complexity', test_label), ('sentence', test_text)]
test = TabularDataset(path = test_filepath, format = 'csv', skip_header = True, fields = test_fields)
test_iter = BucketIterator(test, batch_size = batch_size, device = device, sort_key = lambda x: len(x.sentence), train = False)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.config.num_labels = 1
model.config.output_attentions = True
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.00001)

In [ ]:
def train_bert(model, iterator):
    epoch_loss = 0.0
    model.train()
    for batch in iterator:
        text = batch.sentence
        label = batch.complexity
        optimizer.zero_grad()
        output = model(text)                       
        logits = output.logits[:, : 1]                    
        logits = torch.sigmoid(torch.squeeze(logits))
        attentions = torch.max(torch.max(output.attentions[-1], 3).values, 2).values              # takes maximum attention score from self attentions across heads in last layer of BERT
        try:
            predicted.extend(logits.tolist())
            labels.extend(label.tolist())
            attention_scores.extend(attentions.tolist())
            loss = criterion(label, logits)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        except TypeError:
            pass
    return epoch_loss / len(iterator)

In [ ]:
def test_bert(model, iterator):
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.sentence
            label = batch.complexity
            output = model(text)
            logits = output.logits[:, : 1]                    
            logits = torch.sigmoid(torch.squeeze(logits))
            attentions = torch.max(torch.max(output.attentions[-1], 3).values, 2).values 
            try:
                test_predicted.extend(logits.tolist())
                test_labels.extend(label.tolist())
                test_attention_scores.extend(attentions.tolist())
            except TypeError:
                pass

In [ ]:
def calculate_metrics(y, y_hat):
    vx = y.astype(float)
    vy = y_hat.astype(float)
    pearsonR = np.corrcoef(vx, vy)[0, 1]
    spearmanRho = stats.spearmanr(vx, vy)
    MSE = np.mean((vx - vy) ** 2)
    MAE = np.mean(np.absolute(vx - vy))
    RSquared = (pearsonR ** 2)

    print("Pearson's R: {}".format(pearsonR))
    print("Spearman's rho: {}".format(spearmanRho))
    print("R Squared: {}".format(RSquared))
    print("MSE: {}".format(MSE))
    print("MAE: {}".format(MAE))

In [ ]:
attention_scores = []
labels = []
test_attention_scores = []
test_labels = []
test_predicted = []
print("------BERT Training------")
for epoch in range(num_epochs):
    labels = []
    predicted = []
    attention_scores = []
    train_loss = train_bert(model, train_iter)
    print(f'\t Epoch: {epoch + 1} | Train Loss: {train_loss: }')
    calculate_metrics(np.array(labels), np.array(predicted))
print("------BERT Test----------")
test_bert(model, test_iter)
calculate_metrics(np.array(test_labels), np.array(test_predicted))

In [ ]:
class NN(nn.Module):
  def __init__(self, input_dim):
    super().__init__()
    self.linear1 = nn.Linear(input_dim, 24)
    self.linear2 = nn.Linear(24, 48)
    self.linear3 = nn.Linear(48, 96)
    self.linear4 = nn.Linear(96, 96)
    self.linear5 = nn.Linear(96, 48)
    self.linear6 = nn.Linear(48, 24)
    self.linear7 = nn.Linear(24, 12)
    self.linear8 = nn.Linear(12, 1)

  def forward(self, input):
    out = F.sigmoid(self.linear1(input))
    out = F.sigmoid(self.linear2(out))
    out = F.sigmoid(self.linear3(out))
    out = F.sigmoid(self.linear4(out))
    out = F.sigmoid(self.linear5(out))
    out = F.sigmoid(self.linear6(out))
    out = F.sigmoid(self.linear7(out))
    out = F.sigmoid(self.linear8(out))
    out = torch.squeeze(out)
    return out

In [ ]:
nn_num_epochs = 200
nn_input = torch.tensor(attention_scores, device = device, requires_grad = True)
labels = torch.tensor(labels, device = device, requires_grad = True)
nn_input_test = torch.tensor(test_attention_scores, device = device)

In [ ]:
input_dim = len(attention_scores[0])
nn_model = NN(input_dim)
nn_model.to(device)
nn_criterion = nn.MSELoss()
nn_optimizer = optim.Adam(nn_model.parameters(), lr = 0.00001)
nn_scheduler = torch.optim.lr_scheduler.StepLR(nn_optimizer, step_size = 10, gamma = 1)

In [ ]:
def train_nn(nn_model, input):
    nn_model.train()
    nn_optimizer.zero_grad()
    output = nn_model(input)                       
    loss = nn_criterion(labels, output)
    loss.backward()
    nn_optimizer.step()
    return loss.item()

In [ ]:
def test_nn(nn_model, input):
    nn_model.eval()
    with torch.no_grad():
        output = nn_model(input)
        return output

In [ ]:
for epoch in range(nn_num_epochs):
    nn_train_loss = train_nn(nn_model, nn_input)
    print("Epoch {} : {}".format(epoch + 1, nn_train_loss))
    print("------Metrics for test------")
    output = test_nn(nn_model, nn_input_test)
    calculate_metrics(np.array(test_labels), np.array(output.tolist()))
    nn_scheduler.step()

Only single words have been trained here because doing both single and multi words in same session may lead to CUDA Out Of Memory error, as the attention scores are quite memory intensive to store
